In [11]:
import json 
import os
import numpy as np
import pprint

path = os.getcwd() + "/rohdaten"

entries = []

for file in os.listdir(path):
    with open(path + "/" + file,"r") as f:
        data = json.load(f)
        entries.extend(list(data))

generaldata = []
last_time = entries[2]["time_elapsed"]

counter = 1

ids = []
soa = []
soa_diff = []
decision_time = []
decision_correct = []
word_side = []
response = []

for entry in entries:
    
    if entry.get("soa",None) is not None:

        _soa = entry["soa"]
        _time = entry["time_elapsed"] - last_time
        last_time = entry["time_elapsed"]
        _diff = np.abs(entry["measured_soa"] - _soa)
        _d = int(entry["response_correct"])
        _ws = "l" if entry["probeLeft"] else "r"
        _res = 1 if entry["response"] == "probe" else 0

        ids.append(counter)
        soa.append(_soa)
        soa_diff.append(_time)
        decision_time.append(_diff)
        decision_correct.append(_d)
        word_side.append(_ws)
        response.append(_res)
        # can not use enumerate due to some entries beeing invalid
        counter +=1

    last_time = _time

# garbage now
entries = None

ids = np.array(ids)
soa = np.array(soa)
soa_diff = np.array(soa_diff)
decision_time = np.array(decision_time)
decision_correct = np.array(decision_correct)
word_side = np.array(word_side)
response = np.array(response)

all_soas = np.unique(soa)
all_decisions = np.array(list(zip(soa,decision_correct)))
all_probes = np.array(list(zip(soa,response)))

In [12]:
import plotly.graph_objects as go
data = []

for soa in all_soas:
    index = np.argwhere(all_decisions[:,0]==soa)
    n = index.shape[0]
    right_decisions = np.sum(all_decisions[index,1])
    data.append(right_decisions/n)

fig = go.Figure([go.Bar(x=all_soas, y=data)])
fig.update_layout(
    yaxis_title="Prozentualer Anteil an richtigen Entscheidungen",xaxis_title="SOA Werte"
)
fig.show()

In [13]:
import plotly.graph_objects as go
data = []

for soa in all_soas:
    index = np.argwhere(all_probes[:,0]==soa)
    n = index.shape[0]
    right_decisions = np.sum(1 - all_probes[index,1])
    data.append(right_decisions/n)

fig = go.Figure([go.Bar(x=all_soas, y=data)])
fig.update_layout(
    yaxis_title="Prozentualer Anteil an richtigen Entscheidungen",xaxis_title="SOA Werte"
)
fig.show()